In [178]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

In [179]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [180]:
ruta_train = './data/train_set.csv'
ruta_test = './data/test_set.csv'
ruta_y_pred = './data/sample_submission.csv'
df_train= pd.read_csv(ruta_train, sep = ',', index_col=0)
df_test = pd.read_csv(ruta_test, sep = ',', index_col=0)
df_y_test = pd.read_csv(ruta_y_pred, sep = ',', index_col=0)

C:\Users\Michael_PC\AppData\Local\Temp\ipykernel_7460\225298508.py:4: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train= pd.read_csv(ruta_train, sep = ',', index_col=0)
C:\Users\Michael_PC\AppData\Local\Temp\ipykernel_7460\225298508.py:5: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_test = pd.read_csv(ruta_test, sep = ',', index_col=0)


In [181]:
df_train.head()

,duration,start_time,end_time,start_lat,start_lon,end_lat,end_lon,bike_id,plan_duration,trip_route_category,passholder_type,start_station,end_station
trip_id,,,,,,,,,,,,,
101750280,35,2018-08-07 11:20:00,2018-08-07 11:55:00,33.748920,-118.275192,33.748920,-118.275192,06530,1.0,Round Trip,Walk-up,4127,4127
46560345,32,9/17/2017 17:51,9/17/2017 18:23,34.035679,-118.270813,34.047749,-118.243172,6683,0.0,One Way,Walk-up,3057,3062
120016336,6,2019-04-22 09:22:00,2019-04-22 09:28:00,34.046070,-118.233093,34.047749,-118.243172,06710,30.0,One Way,Monthly Pass,3022,3062
129547190,138,9/22/2019 11:27,9/22/2019 13:45,34.062580,-118.290092,34.059689,-118.294662,17068,1.0,One Way,One Day Pass,4304,4311
136619463,14,1/31/2020 17:11,1/31/2020 17:25,34.026291,-118.277687,34.021660,-118.278687,18841,30.0,One Way,Monthly Pass,4266,4443


In [182]:
df_test.head()

,duration,start_time,end_time,start_lat,start_lon,end_lat,end_lon,bike_id,trip_route_category,start_station,end_station
trip_id,,,,,,,,,,,
17059130,12,1/1/2017 0:24,1/1/2017 0:36,34.058319,-118.246094,34.058319,-118.246094,6351,Round Trip,3028,3028
17059129,17,1/1/2017 0:28,1/1/2017 0:45,34.049980,-118.247162,34.043732,-118.260139,5836,One Way,3027,3018
17059126,20,1/1/2017 0:39,1/1/2017 0:59,34.063389,-118.236160,34.044159,-118.251579,6529,One Way,3066,3055
17059125,12,1/1/2017 0:43,1/1/2017 0:55,34.048851,-118.246422,34.050140,-118.233238,6029,One Way,3029,3079
17061379,48,1/1/2017 0:56,1/1/2017 1:44,34.049198,-118.252831,34.049198,-118.252831,6680,Round Trip,3063,3063


In [183]:
df_train.shape, df_test.shape

((700000, 13), (569886, 11))

#### Columnas que están en el df_train y que no están en df_test

In [184]:
set(df_train.columns) - set(df_test.columns)

{'passholder_type', 'plan_duration'}

### adicionando la variable objetivo al conjunto de testeo

In [185]:
df_test = pd.merge(df_test, df_y_test, on = 'trip_id', how = 'inner')
df_test.head()

,duration,start_time,end_time,start_lat,start_lon,end_lat,end_lon,bike_id,trip_route_category,start_station,end_station,passholder_type
trip_id,,,,,,,,,,,,
17059130,12,1/1/2017 0:24,1/1/2017 0:36,34.058319,-118.246094,34.058319,-118.246094,6351,Round Trip,3028,3028,Monthly Pass
17059129,17,1/1/2017 0:28,1/1/2017 0:45,34.049980,-118.247162,34.043732,-118.260139,5836,One Way,3027,3018,Monthly Pass
17059126,20,1/1/2017 0:39,1/1/2017 0:59,34.063389,-118.236160,34.044159,-118.251579,6529,One Way,3066,3055,Monthly Pass
17059125,12,1/1/2017 0:43,1/1/2017 0:55,34.048851,-118.246422,34.050140,-118.233238,6029,One Way,3029,3079,Monthly Pass
17061379,48,1/1/2017 0:56,1/1/2017 1:44,34.049198,-118.252831,34.049198,-118.252831,6680,Round Trip,3063,3063,Monthly Pass


### Tratamiento de la variable bike_id

In [186]:
non_numeric_rows = df_train[
    pd.to_numeric(df_train['bike_id'], errors='coerce').isna()]

print("Non-numeric rows:")
non_numeric_rows.shape
set(non_numeric_rows['bike_id'])

Non-numeric rows:


{'15111a',
 '16015a',
 'Dock Block 1',
 'Dock Block 2',
 'Rblock',
 'TBlocker_Slash',
 '_Test_RFID'}

In [187]:
dict_train = {'15111a':111, '16015a':222, 'Dock Block 1': 333,
              'Dock Block 2':444, 'Rblock':555,
              'TBlocker_Slash':666,
              '_Test_RFID':777}
df_train['bike_id'] = df_train['bike_id'].replace(dict_train).astype(int)

In [188]:
non_numeric_rows = df_test[
    pd.to_numeric(df_test['bike_id'], errors='coerce').isna()]

print("Non-numeric rows:")
non_numeric_rows.shape
set(non_numeric_rows['bike_id'])

Non-numeric rows:


{'15111a',
 '15229a',
 '15241b',
 '16015a',
 'Dock Block 1',
 'Dock Block 2',
 'Rblock',
 'TBlocker_Slash',
 '_Test_RFID'}

In [189]:
dict_test = {'15111a':111, '16015a':222, 'Dock Block 1': 333,
              'Dock Block 2':444, 'Rblock':555,
              'TBlocker_Slash':666,
              '_Test_RFID':777, '15229a':888, '15241b':999
              }
df_test['bike_id'] = df_test['bike_id'].replace(dict_test).astype(int)
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 569886 entries, 17059130 to 179492474
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   duration             569886 non-null  int64  
 1   start_time           569886 non-null  object 
 2   end_time             569886 non-null  object 
 3   start_lat            565264 non-null  float64
 4   start_lon            565264 non-null  float64
 5   end_lat              554995 non-null  float64
 6   end_lon              554995 non-null  float64
 7   bike_id              569886 non-null  int64  
 8   trip_route_category  569886 non-null  object 
 9   start_station        569886 non-null  int64  
 10  end_station          569886 non-null  int64  
 11  passholder_type      569886 non-null  object 
dtypes: float64(4), int64(4), object(4)
memory usage: 56.5+ MB


#### Uniendo los dataset df_train, df_test con el fin de realizar el análisis exploratorio de datos

In [190]:
df = pd.concat([df_train, df_test], axis = 0, join = 'outer')
print(df.shape)
df.head()

(1269886, 13)


,duration,start_time,end_time,start_lat,start_lon,end_lat,end_lon,bike_id,plan_duration,trip_route_category,passholder_type,start_station,end_station
trip_id,,,,,,,,,,,,,
101750280,35,2018-08-07 11:20:00,2018-08-07 11:55:00,33.748920,-118.275192,33.748920,-118.275192,6530,1.0,Round Trip,Walk-up,4127,4127
46560345,32,9/17/2017 17:51,9/17/2017 18:23,34.035679,-118.270813,34.047749,-118.243172,6683,0.0,One Way,Walk-up,3057,3062
120016336,6,2019-04-22 09:22:00,2019-04-22 09:28:00,34.046070,-118.233093,34.047749,-118.243172,6710,30.0,One Way,Monthly Pass,3022,3062
129547190,138,9/22/2019 11:27,9/22/2019 13:45,34.062580,-118.290092,34.059689,-118.294662,17068,1.0,One Way,One Day Pass,4304,4311
136619463,14,1/31/2020 17:11,1/31/2020 17:25,34.026291,-118.277687,34.021660,-118.278687,18841,30.0,One Way,Monthly Pass,4266,4443


#### Datos nulos

In [191]:
df_train.shape

(700000, 13)

In [192]:
df_train.isnull().sum()/df_train.shape[0]

duration               0.000000
start_time             0.000000
end_time               0.000000
start_lat              0.007947
start_lon              0.007947
end_lat                0.026534
end_lon                0.026534
bike_id                0.000000
plan_duration          0.000297
trip_route_category    0.000000
passholder_type        0.003680
start_station          0.000000
end_station            0.000000
dtype: float64

### Borramos los datos nulos del conjunto de entrenamiento

Dado que el porcentaje de datos nulos no supera el 2% de nulos por columna con datos nulos

In [193]:
df_train_sin_nulos = df_train.dropna()
df_train_sin_nulos.shape

(675626, 13)

In [194]:
df_train_sin_nulos.isnull().sum()

duration               0
start_time             0
end_time               0
start_lat              0
start_lon              0
end_lat                0
end_lon                0
bike_id                0
plan_duration          0
trip_route_category    0
passholder_type        0
start_station          0
end_station            0
dtype: int64

#### Porcentaje de datos eliminados en el conjunto de entrenamiento

In [195]:
(df_train.shape[0] - df_train_sin_nulos.shape[0])/df_train.shape[0]

0.03482

In [196]:
df_test.isnull().sum()/df_test.shape[0]

duration               0.00000
start_time             0.00000
end_time               0.00000
start_lat              0.00811
start_lon              0.00811
end_lat                0.02613
end_lon                0.02613
bike_id                0.00000
trip_route_category    0.00000
start_station          0.00000
end_station            0.00000
passholder_type        0.00000
dtype: float64

### Borrando los datos nulos del conjunto de testeo

In [197]:
df_test_sin_nulos = df_test.dropna()

In [198]:
df_test_sin_nulos.isnull().sum()

duration               0
start_time             0
end_time               0
start_lat              0
start_lon              0
end_lat                0
end_lon                0
bike_id                0
trip_route_category    0
start_station          0
end_station            0
passholder_type        0
dtype: int64

### Porcentaje de filas eliminadas en el conjunto de entrenamiento

In [199]:
(df_test.shape[0] - df_test_sin_nulos.shape[0])/df_test.shape[0]

0.030646480173227627

In [200]:
df.isnull().sum()/df.shape[0]

duration               0.000000
start_time             0.000000
end_time               0.000000
start_lat              0.008020
start_lon              0.008020
end_lat                0.026353
end_lon                0.026353
bike_id                0.000000
plan_duration          0.448933
trip_route_category    0.000000
passholder_type        0.002029
start_station          0.000000
end_station            0.000000
dtype: float64

#### Convertir las variables start_time y end_time a fecha

#### Preprocesando la variable start_time y end_time

In [201]:
df_train['start_time'] = pd.to_datetime(df_train['start_time'],
                                        format = 'mixed')
df_train['end_time'] = pd.to_datetime(df_train['end_time'],
                                        format = 'mixed')
df_test['start_time'] = pd.to_datetime(df_test['start_time'],
                                        format = 'mixed')
df_test['end_time'] = pd.to_datetime(df_test['end_time'],
                                        format = 'mixed')
df['start_time'] = pd.to_datetime(df['start_time'],
                                        format = 'mixed')
df['end_time'] = pd.to_datetime(df['end_time'],
                                        format = 'mixed')
df_train_sin_nulos['start_time'] = pd.to_datetime(df_train_sin_nulos['start_time'],
                                        format = 'mixed')
df_train_sin_nulos['end_time'] = pd.to_datetime(df_train_sin_nulos['end_time'],
                                        format = 'mixed')
df_test_sin_nulos['start_time'] = pd.to_datetime(df_test_sin_nulos['start_time'],
                                        format = 'mixed')
df_test_sin_nulos['end_time'] = pd.to_datetime(df_test_sin_nulos['end_time'],
                                        format = 'mixed')

C:\Users\Michael_PC\AppData\Local\Temp\ipykernel_7460\596530330.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_sin_nulos['start_time'] = pd.to_datetime(df_train_sin_nulos['start_time'],
C:\Users\Michael_PC\AppData\Local\Temp\ipykernel_7460\596530330.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_sin_nulos['end_time'] = pd.to_datetime(df_train_sin_nulos['end_time'],
C:\Users\Michael_PC\AppData\Local\Temp\ipykernel_7460\596530330.py:17: SettingWithCopyWarning: 
A value is trying 

#### Adicionando las variables de día, mes, hora para cada una de las variables "start_time" y "end_time"

Se crean estas variables con el fin de realizar un mejor análisis exploratorio de datos

In [202]:
# Extrayendo los componente de 'start_time'
df['start_day'] = df['start_time'].dt.day
df['start_month'] = df['start_time'].dt.month
df['start_year'] = df['start_time'].dt.year
df['start_hour'] = df['start_time'].dt.hour
df['start_minute'] = df['start_time'].dt.minute
df['start_second'] = df['start_time'].dt.second

# Extrayendo los componentes de 'end_time'
df['end_day'] = df['end_time'].dt.day
df['end_month'] = df['end_time'].dt.month
df['end_year'] = df['end_time'].dt.year
df['end_hour'] = df['end_time'].dt.hour
df['end_minute'] = df['end_time'].dt.minute
df['end_second'] = df['end_time'].dt.second

### df_train
# Extrayendo los componente de 'start_time'
df_train['start_day'] = df_train['start_time'].dt.day
df_train['start_month'] = df_train['start_time'].dt.month
df_train['start_year'] = df_train['start_time'].dt.year
df_train['start_hour'] = df_train['start_time'].dt.hour
df_train['start_minute'] = df_train['start_time'].dt.minute
df_train['start_second'] = df_train['start_time'].dt.second

# Extrayendo los componentes de 'end_time'
df_train['end_day'] = df_train['end_time'].dt.day
df_train['end_month'] = df_train['end_time'].dt.month
df_train['end_year'] = df_train['end_time'].dt.year
df_train['end_hour'] = df_train['end_time'].dt.hour
df_train['end_minute'] = df_train['end_time'].dt.minute
df_train['end_second'] = df_train['end_time'].dt.second

### df_test
# Extrayendo los componente de 'start_time'
df_test['start_day'] = df_test['start_time'].dt.day
df_test['start_month'] = df_test['start_time'].dt.month
df_test['start_year'] = df_test['start_time'].dt.year
df_test['start_hour'] = df_test['start_time'].dt.hour
df_test['start_minute'] = df_test['start_time'].dt.minute
df_test['start_second'] = df_test['start_time'].dt.second

# Extrayendo los componentes de 'end_time'
df_test['end_day'] = df_test['end_time'].dt.day
df_test['end_month'] = df_test['end_time'].dt.month
df_test['end_year'] = df_test['end_time'].dt.year
df_test['end_hour'] = df_test['end_time'].dt.hour
df_test['end_minute'] = df_test['end_time'].dt.minute
df_test['end_second'] = df_test['end_time'].dt.second

### df_train_sin_nulos
# Extrayendo los componente de 'start_time'
df_train_sin_nulos['start_day'] = df_train_sin_nulos['start_time'].dt.day
df_train_sin_nulos['start_month'] = df_train_sin_nulos['start_time'].dt.month
df_train_sin_nulos['start_year'] = df_train_sin_nulos['start_time'].dt.year
df_train_sin_nulos['start_hour'] = df_train_sin_nulos['start_time'].dt.hour
df_train_sin_nulos['start_minute'] = df_train_sin_nulos['start_time'].dt.minute
df_train_sin_nulos['start_second'] = df_train_sin_nulos['start_time'].dt.second

# Extrayendo los componentes de 'end_time'
df_train_sin_nulos['end_day'] = df_train_sin_nulos['end_time'].dt.day
df_train_sin_nulos['end_month'] = df_train_sin_nulos['end_time'].dt.month
df_train_sin_nulos['end_year'] = df_train_sin_nulos['end_time'].dt.year
df_train_sin_nulos['end_hour'] = df_train_sin_nulos['end_time'].dt.hour
df_train_sin_nulos['end_minute'] = df_train_sin_nulos['end_time'].dt.minute
df_train_sin_nulos['end_second'] = df_train_sin_nulos['end_time'].dt.second

### df_test_sin_nulos
# Extrayendo los componente de 'start_time'
df_test_sin_nulos['start_day'] = df_test_sin_nulos['start_time'].dt.day
df_test_sin_nulos['start_month'] = df_test_sin_nulos['start_time'].dt.month
df_test_sin_nulos['start_year'] = df_test_sin_nulos['start_time'].dt.year
df_test_sin_nulos['start_hour'] = df_test_sin_nulos['start_time'].dt.hour
df_test_sin_nulos['start_minute'] = df_test_sin_nulos['start_time'].dt.minute
df_test_sin_nulos['start_second'] = df_test_sin_nulos['start_time'].dt.second

# Extrayendo los componentes de 'end_time'
df_test_sin_nulos['end_day'] = df_test_sin_nulos['end_time'].dt.day
df_test_sin_nulos['end_month'] = df_test_sin_nulos['end_time'].dt.month
df_test_sin_nulos['end_year'] = df_test_sin_nulos['end_time'].dt.year
df_test_sin_nulos['end_hour'] = df_test_sin_nulos['end_time'].dt.hour
df_test_sin_nulos['end_minute'] = df_test_sin_nulos['end_time'].dt.minute
df_test_sin_nulos['end_second'] = df_test_sin_nulos['end_time'].dt.second

C:\Users\Michael_PC\AppData\Local\Temp\ipykernel_7460\410442427.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_sin_nulos['start_day'] = df_train_sin_nulos['start_time'].dt.day
C:\Users\Michael_PC\AppData\Local\Temp\ipykernel_7460\410442427.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_sin_nulos['start_month'] = df_train_sin_nulos['start_time'].dt.month
C:\Users\Michael_PC\AppData\Local\Temp\ipykernel_7460\410442427.py:55: SettingWithCopyWarning: 
A value is trying to be set on

### Guardando los datos en archivos .csv

In [203]:
ruta_df = './data/df_set.csv'
ruta_train = './data/train_set_prepocesado.csv'
ruta_test = './data/test_set_prepocesado.csv'
ruta_train_sin_nulos = './data/train_sin_nulos.csv'
ruta_test_sin_nulos = './data/test_sin_nulos.csv'
df_train_sin_nulos.to_csv(ruta_train_sin_nulos, sep = ',')
df_test_sin_nulos.to_csv(ruta_test_sin_nulos, sep = ',')
df_train.to_csv(ruta_train, sep = ',')
df_test.to_csv(ruta_test, sep = ',')
df.to_csv(ruta_df, sep = ',')